In [5]:
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

torch.manual_seed(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root = "MNIST_data/", train = True,
                         transform = transforms.ToTensor(),
                         download = True)
mnist_test = dsets.MNIST(root = "MNIST_data/", train = False,
                        transform = transforms.ToTensor(),
                        download = True)

data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True)

# CNN Model
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self._build_net()
        
    def _build_net(self):
        self.keep_prob = 0.7

        # L1 ImgIn shape = (?, 28, 28, 1)
        # Conv           = (?, 28, 28, 32)
        # Pool           = (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size= 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
        torch.nn.Dropout(p = 1 - self.keep_prob))

        # L2 ImgIn shape = (?, 14, 14, 32)
        # Conv           = (?, 14, 14, 64)
        # Pool           = (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
        torch.nn.Dropout(p = 1 - self.keep_prob))

        # L3 ImgIn shape = (?, 7, 7, 64)
        # Conv           = (?, 7, 7, 128)
        # Pool           = (?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1),
        torch.nn.Dropout(p = 1 - self.keep_prob))

        # L4 FC 4 * 4* 128 inputs -> 625 outputs
        self.keep_prob = 0.5
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias = True)
        torch.nn.init.xavier_uniform(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
        self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p = 1 - self.keep_prob))

        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform(self.fc2.weight)

        self.criterion = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr = learning_rate)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # Flatten
        out = self.fc1(out)
        out = self.fc2(out)

        return out

    def predict(self, x):
        self.eval()

        return self.forward(x)

    def get_accuracy(self, x, y):
        prediction = self.predict(x)
        correct_prediction = (torch.max(prediction.data, 1)[1] == y.data)
        self.accuracy = correct_prediction.float().mean()

        return self.accuracy

    def train_model(self, x, y):
        self.train()
        self.optimizer.zero_grad()
        hypothesis = self.forward(x)
        self.cost = self.criterion(hypothesis, y)
        self.cost.backward()
        self.optimizer.step()

        return self.cost
        
model = CNN()

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(mnist_train)
    
    for i, (batch_xs, batch_ys) in enumerate(data_loader):
        X = Variable(batch_xs) # 이미지가 이미 28 * 28 사이즈로 되어 있다.
        Y = Variable(batch_ys)
        
        cost = model.train_model(X, Y)
        
        avg_cost += cost.data / total_batch
        
    print("[Epoch : {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost))
    
print("Learning Finished!")

X_test = Variable(mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float())
Y_test = Variable(mnist_test.test_labels)

print("Accuracy :", model.get_accuracy(X_test, Y_test))

/Users/wonjuson/anaconda2/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/Users/wonjuson/anaconda2/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:69: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


Learning started. It takes sometime.
[Epoch :    1] cost = 0.00286720716
[Epoch :    2] cost = 0.000857745646
[Epoch :    3] cost = 0.000683243328
[Epoch :    4] cost = 0.00056240306
[Epoch :    5] cost = 0.000528416596
[Epoch :    6] cost = 0.000481130584
[Epoch :    7] cost = 0.000440739997
[Epoch :    8] cost = 0.000426688639
[Epoch :    9] cost = 0.000389817636
[Epoch :   10] cost = 0.00039449922
[Epoch :   11] cost = 0.000359835831
[Epoch :   12] cost = 0.000356865203
[Epoch :   13] cost = 0.000342590967
[Epoch :   14] cost = 0.000327843707
[Epoch :   15] cost = 0.000344855478
Learning Finished!


/Users/wonjuson/anaconda2/envs/pytorch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/wonjuson/anaconda2/envs/pytorch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy : tensor(0.7255)
